In [1]:
# 3, 앵커 목소리가 기자 목소리로 전환될 때 video cut (time stamp)

# [ 마저 처리할 일들 ]
# str -> time 형태로 바꾸기
# 함수 형태로 만들어주기
# 전체 동영상 파일 대상으로 일반화 작업할 수 있도록 만들어주기

In [2]:
# !pip install git+https://github.com/pyannote/pyannote-audio

In [3]:
import torch
import pyannote
from pyannote.database import get_protocol
from pyannote.database import FileFinder
from pyannote.audio.utils.signal import Binarize

import os
import re
import moviepy.editor as mp
import time

C:\Users\YangGang\anaconda3\lib\site-packages\pyannote\database\database.py:51: UserWarning: Ignoring deprecated 'preprocessors' argument in AMI.__init__. Pass it to 'get_protocol' instead.
  warnings.warn(msg)


In [4]:
# Speech activity detection
sad = torch.hub.load('pyannote/pyannote-audio', 'sad_ami')

Using cache found in C:\Users\YangGang/.cache\torch\hub\pyannote_pyannote-audio_master


In [5]:
preprocessors = {'audio': FileFinder()}
protocol = get_protocol('AMI.SpeakerDiarization.MixHeadset',
                        preprocessors=preprocessors)

# 연습 예제
test_file = next(protocol.test())

In [6]:
MOV_Directory = './MOV'
VideoList = os.listdir(MOV_Directory)

WAV_Directory = './WAV_streo'

for i in VideoList:
    i2 = MOV_Directory + '/' + i
    clip = mp.VideoFileClip(i2)
    
    sound_file_name = re.sub(pattern='mp4', repl='wav', string=i)
    sound_file_name = WAV_Directory + '/' +sound_file_name
    clip.audio.write_audiofile(sound_file_name)

MoviePy - Writing audio in ./WAV_streo/readnaversid1750oid448aid0000195268.wav


MoviePy - Done.
MoviePy - Writing audio in ./WAV_streo/readnaversid1750oid448aid0000195576.wav


MoviePy - Done.
MoviePy - Writing audio in ./WAV_streo/readnaversid1750oid448aid0000196603.wav


MoviePy - Done.
MoviePy - Writing audio in ./WAV_streo/video.wav


MoviePy - Done.


In [7]:
# test_file_1 = {'uri':'video', 'audio':'./WAV_streo/readnaversid1750oid448aid0000196603.wav'}
test_file_2 = {'audio':'./WAV_streo/video.wav'}

In [8]:
sad_scores = sad(test_file_2)

In [9]:
binarize = Binarize(offset=0.52, onset=0.52, log_scale=True,
                   min_duration_off=0.1, min_duration_on=0.1)

In [10]:
speech = binarize.apply(sad_scores, dimension=1)

# 0번째 인덱스의 끝 부분 : 앵커에서 기자로 넘어가는 시점
str(speech)

'[[ 00:00:00.010 -->  00:00:19.104]\n [ 00:00:21.195 -->  00:01:57.698]]'

In [11]:
a = list(speech)
b = str(a[0])

c = re.sub(pattern='\-\-\>', repl='', string=b)
c = re.sub(pattern='\[', repl='', string=c)
c = re.sub(pattern='\]', repl='', string=c)
c = str.split(c)

c

['00:00:00.010', '00:00:19.104']